In [1]:
# Imports 
import mindwave
import time
import pandas as pd
import numpy as np
import random 
from pygame import mixer
import threading
import tkinter as tk
from tkinter import ttk

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
global filename

In [3]:
ls =!ls /dev
ls

['HAX',
 'afsc_type5',
 'auditpipe',
 'auditsessions',
 'autofs',
 'autofs_control',
 'autofs_homedirmounter',
 'autofs_notrigger',
 'autofs_nowait',
 'bpf0',
 'bpf1',
 'bpf2',
 'bpf3',
 'console',
 'cu.Bluetooth-Incoming-Port',
 'cu.BoseBose-SPPDev-1',
 'cu.BoseBose-SPPDev-7',
 'cu.BoseColorIISoundLink-SP-7',
 'cu.BoseColorIISoundLink-SP-8',
 'cu.ElizabethsIPhone-Wirele',
 'cu.MindWaveMobile-SerialPo',
 'disk0',
 'disk0s1',
 'disk0s2',
 'disk1',
 'disk1s1',
 'disk1s2',
 'disk1s3',
 'disk1s4',
 'disk1s5',
 'dtrace',
 'dtracehelper',
 'fbt',
 'fd',
 'fsevents',
 'io8log',
 'io8logmt',
 'io8logtemp',
 'klog',
 'lockstat',
 'machtrace',
 'null',
 'oslog',
 'oslog_stream',
 'pf',
 'pfm',
 'profile',
 'ptmx',
 'ptyp0',
 'ptyp1',
 'ptyp2',
 'ptyp3',
 'ptyp4',
 'ptyp5',
 'ptyp6',
 'ptyp7',
 'ptyp8',
 'ptyp9',
 'ptypa',
 'ptypb',
 'ptypc',
 'ptypd',
 'ptype',
 'ptypf',
 'ptyq0',
 'ptyq1',
 'ptyq2',
 'ptyq3',
 'ptyq4',
 'ptyq5',
 'ptyq6',
 'ptyq7',
 'ptyq8',
 'ptyq9',
 'ptyqa',
 'ptyqb',
 'ptyq

### GUI Functions

In [4]:
def play_music():
    try:
        paused
    except NameError: 
        try:
            mixer.music.load('data/' + filename)
            mixer.music.play()
            show_details()
            
        except:
            print('File not found')
    else:
        mixer.music.unpause()
        paused = False 

In [5]:
paused = False 
def pause_music():
    global paused
    paused = True
    mixer.music.pause()

#### Load Data

In [6]:
# Music Options
df_music = pd.read_csv('Data/Trimmed/music_categories_trimmed.csv')

### Neurosky Functions

In [30]:
def calcAvg(time_limit,medprogbar,attprogbar):
    attention_total = 0
    meditation_total = 0
    time_elapsed = 0
    while time_elapsed <= time_limit:
        attention_total += headset.attention
        meditation_total += headset.meditation
        win.update()
        medprogbar['value'] = headset.meditation
        attprogbar['value'] = headset.attention
        mins, secs = divmod(time_elapsed, 60)
        mins = int(round(mins))
        secs = int(round(secs))
        curtimeformat = '{:02d}:{:02d}'.format(mins, secs)
        currenttimelabel['text'] = "Current Time: " + curtimeformat
        time.sleep(1)
        time_elapsed += 1
    attention_avg = attention_total/time_elapsed
    meditation_avg = meditation_total/time_elapsed
    return attention_avg, meditation_avg

In [8]:
def selectSong(df_music, attention_avg, meditation_avg):
    if (attention_avg >= 50 and meditation_avg > 50):
        song_choice = 1 #High A, High M song 
    elif (attention_avg > 50 and meditation_avg <= 50):
        song_choice = 2 #High A, Low M song 
    elif (attention_avg < 50 and meditation_avg >= 50):
        song_choice = 3 #Low A, High M song 
    elif (attention_avg <= 50 and meditation_avg < 50):
        song_choice = 4 #Low A, Low M song 
    else:
        song_choice = 0

    category_df = df_music.loc[df_music['Category']==song_choice]
    indexes = category_df.index
    randIdx = random.choice(indexes) 
    randSong = category_df.loc[randIdx]['Song_Name']
    songLength = category_df.loc[randIdx]['Song_Length']
    return randSong, songLength

### SETTING UP GUI

In [9]:
mixer.init()

In [77]:
win = tk.Tk()

win.geometry('400x135')
win.configure(background='gray')
win.title("Music Productivity")

style = ttk.Style()
style.theme_use('clam')
style.configure("blue.Horizontal.TProgressbar", foreground='blue', background='blue')
style.configure("red.Horizontal.TProgressbar", foreground='red', background='red')

# labels for attention and meditation levels 
attentionlabel = tk.Label(win, text='Attention Level: ', bg = 'gray', font="Verdana 12 italic")
attentionlabel.pack()
attentionlabel.grid
attentionlabel.place(x=10, y=0)

meditationlabel = tk.Label(win, text='Meditation Level: ', bg = 'gray', font="Verdana 12 italic")
meditationlabel.pack()
meditationlabel.grid
meditationlabel.place(x=210, y=0)

# labels for attention and meditation total average 
attentionlabel = tk.Label(win, text='Attention Average: ', bg = 'gray', font="Verdana 10 italic")
attentionlabel.pack()
attentionlabel.grid
attentionlabel.place(x=10, y=37)

meditationlabel = tk.Label(win, text='Meditation Average: ', bg = 'gray', font="Verdana 10 italic")
meditationlabel.pack()
meditationlabel.grid
meditationlabel.place(x=210, y=37)

# Creating progress bars for attention, meditation, and song progression 
medprogbar = ttk.Progressbar(win,style="blue.Horizontal.TProgressbar",orient='horizontal',length = 180, mode = 'determinate', maximum = 100)
medprogbar.pack()
medprogbar.place(x=210, y=20)

attprogbar = ttk.Progressbar(win,style="red.Horizontal.TProgressbar",orient='horizontal',length = 180, mode = 'determinate', maximum = 100)
attprogbar.pack()
attprogbar.place(x=10, y=20)


# labels for length of song and current play time placed on either side of the song progress bar
lengthlabel = tk.Label(win, text='Song Length: 00:00', bg = 'gray', font="Verdana 12 italic")
lengthlabel.pack()
lengthlabel.grid
lengthlabel.place(x=140, y=80)

currenttimelabel = tk.Label(win, text='Current Time: 00:00', bg = 'gray', font="Verdana 12 italic")
currenttimelabel.pack()
currenttimelabel.grid
currenttimelabel.place(x=10, y=80)

# Create play button and label for current song playing 
playPhoto = tk.PhotoImage(file = 'play.png')
playBtn = tk.Button(win, image = playPhoto, command = play_music)
playBtn.pack()
playBtn.grid()
playBtn.place(x=297, y=65)

playText = tk.Label(win, text= 'Playing: ' + 'N/A', bg= "gray",font="Verdana 12 italic")
playText.pack()
playText.grid
playText.place(x=10, y=60)

# Create pause button and label for queued song 
pausePhoto = tk.PhotoImage(file= 'pause.png')
pauseBtn = tk.Button(win, image = pausePhoto, command= pause_music)
pauseBtn.pack()
pauseBtn.grid
pauseBtn.place(x=343, y=65)

statusbar = tk.Label(win, text="Welcome to Music Productivity",  bg= "lightgray",font="Verdana 12 italic", relief= tk.SUNKEN, anchor= tk.W)
statusbar.pack(side= tk.BOTTOM, fill= tk.X)

headset = mindwave.Headset('/dev/cu.MindWaveMobile-SerialPo','MMPPHB2013')
time.sleep(2)
headset.autoconnect()
playlist = []
count = 1

# __________HEADSET STUFF_____________
while headset.status != ('connected'):
    time.sleep(1)
    if headset.status == None:
        if(headset.attention != 0 or headset.meditation != 0):
            # Choosing the First Song
            if (count == 1):
                print('Choosing First Song')
                time_limit = 10
                t_end = time.time() + time_limit
                while (time.time() < t_end):
                    attention_avg, meditation_avg = calcAvg(time_limit,medprogbar,attprogbar)
                print(attention_avg, meditation_avg)
                firstSong, songLength = selectSong(df_music, attention_avg, meditation_avg)
                print('First Song: ' + str(firstSong))
                
                # Playing the first song 
                mins, secs = divmod(songLength, 60)
                mins = int(round(mins))
                secs = int(round(secs))
                win.update()
                playText['text'] = "Playing: " + firstSong
                timeformat = '{:02d}:{:02d}'.format(mins, secs)
                lengthlabel['text'] = "Song Length: " + timeformat
                firstSong = 'Data/Trimmed/' +  str(firstSong)
                mixer.music.load(firstSong)
                mixer.music.play()
                win.update()
                
            # Choosing all the other songs    
            t_start = time.time()
            while (time.time() - t_start < songLength - 1):
                attention_avg, meditation_avg = calcAvg(songLength - 1,medprogbar,attprogbar)
            print(attention_avg, meditation_avg)
            nextSong, songLength = selectSong(df_music, attention_avg, meditation_avg)
            print('Next Song: ' + str(nextSong))
            mins, secs = divmod(songLength, 60)
            mins = int(round(mins))
            secs = int(round(secs))
            playText['text'] = "Playing: " + nextSong
            timeformat = '{:02d}:{:02d}'.format(mins, secs)
            lengthlabel['text'] = "Song Length: " + timeformat
            nextSong = 'Data/Trimmed/' +  str(nextSong)
            playlist.append(nextSong)
            mixer.music.load(nextSong)
            mixer.music.queue(playlist.pop(0))
            mixer.music.play()
            win.update()
            count += 1 
        else:
            print('Still Trying to Connect')
            
win.mainloop()

Choosing First Song
(48, 40)
First Song: TheImperialMarch trimmed.mp3
(44, 60)
Next Song: 02 Strange Fruit trimmed.mp3


TclError: invalid command name ".4622202856"

In [76]:
win.mainloop()

In [74]:
mixer.music.pause()